# **Normalized Difference Bare Soil Index (NDBSI) Dubai** 

# Urban Evolution of Dubai - A Comparative Remote Sensing Analysis from 1995 to 2024

**Erik Ashkinadze (erik.ashkinadze@ruhr-uni-bochum.de)**

**Devon Klör (devon.kloer@ruhr-uni-bochum.de)**

**Course:** Geographic Information Systems (GIS I): Databases and Programming

**Professor:** Jun.-Prof. Dr. Andreas Rienow

**References:** Liu, Y.; Meng, Q.; Zhang, L.; Wu, C. (2022): NDBSI: A normalized difference bare soil index for remote sensing to improve bare soil mapping accuracy in urban and rural areas. In: Catena, 214: 1-11. DOI:  https://doi.org/10.1016/j.catena.2022.106265

**Repository:** GIS 1 Course "03_Create_landsat_timelapse_byQuishengWu"

In [ ]:
import ee
import geemap

# Authenticate and Initialize Earth Engine with your project ID
# ee.Authenticate()
ee.Initialize(project='ee-dkloer01')

# Shapefile einladen (Pfad zur Datei anpassen)
shapefile_path = "./Dubai_Shapes/Dubai.shp"
dubai = geemap.shp_to_ee(shapefile_path)

# Loading the Image Collections from Landsat 5, 7 and 8
# Landsat 5 for 1995 (TM Sensor)
ls_ic_1995 = ee.ImageCollection('LANDSAT/LT05/C02/T1_L2') \
    .filterDate('1995-01-01', '1995-12-31') \
    .filterBounds(dubai) \
    .median()

# Landsat 7 for 2005 (TM Sensor)
ls_ic_2005 = ee.ImageCollection('LANDSAT/LE07/C02/T1_L2') \
    .filterDate('2005-01-01', '2005-12-31') \
    .filterBounds(dubai) \
    .median()

# Landsat 8 for 2015 (OLI Sensor)
ls_ic_2015 = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
    .filterDate('2015-01-01', '2015-12-31') \
    .filterBounds(dubai) \
    .median()

# Landsat 8 for 2025 (OLI Sensor)
ls_ic_2024 = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
    .filterDate('2024-01-01', '2024-12-31') \
    .filterBounds(dubai) \
    .median()

# Calculation of the Normalized Difference Bare Soil Index (NDBSI)
# Using band 5 (SWIR1), band 4 (NIR) and band 1 (Blue) for historic analysis with Landsat 5 & 7
# Using band 6 (SWIR1), band 5 (NIR) and band 2 (Blue) for current analysis with Landsat 8
def calculate_ndbsi(image, year):
    if year <= 2005:
        # Landsat 5 & 7
        swir1 = image.select('SR_B5')
        nir = image.select('SR_B4')
        blue = image.select('SR_B1')
    else:
        # Landsat 8
        swir1 = image.select('SR_B6')
        nir = image.select('SR_B5')
        blue = image.select('SR_B2')

    # NDBSI formula (orientated on Liu et al. 2022)
    numerator = (swir1.subtract(blue))
    denominator = (swir1.add(blue))
    ndbsi = numerator.divide(denominator)
    return ndbsi.addBands(ndbsi.rename('NDBSI'))

# Calculate NDBSI for each year
ndbsi_bands_1995 = calculate_ndbsi(ls_ic_1995, 1995)
ndbsi_1995 = ndbsi_bands_1995.select('NDBSI').clip(dubai)

ndbsi_bands_2005 = calculate_ndbsi(ls_ic_2005, 2005)
ndbsi_2005 = ndbsi_bands_2005.select('NDBSI').clip(dubai)

ndbsi_bands_2015 = calculate_ndbsi(ls_ic_2015, 2015)
ndbsi_2015 = ndbsi_bands_2015.select('NDBSI').clip(dubai)

ndbsi_bands_2024 = calculate_ndbsi(ls_ic_2024, 2024)
ndbsi_2024 = ndbsi_bands_2024.select('NDBSI').clip(dubai)

# BSI Change Detection
ndbsi_change = ndbsi_2024.subtract(ndbsi_1995).rename('NDBSI Change').clip(dubai)

# This color palette is designed to visualize the Normalized Difference Bare Soil Index (NDBSI).
ndbsi_palette = [
    '#2900fa',  
    '#1a9850',  
    '#ffffbf',  
    '#fb9c14',  
    '#d73027'   
]

# This color palette is designed to visualize changes in the NDBSI over time.
ndbsi_change_palette = [
    '#fb0114',  
    '#fbfdff', 
    '#039d02'   
]

# These visualization settings define how the NDBSI values are mapped to the color palette
ndbsi_vis = {
    'min': -0.3, # The minimum NDBSI value to be displayed in the visualization. Values below this will be colored with the lowest end of the palette
    'max': 0.3, # The maximum NDBSI value to be displayed in the visualization. Values above this will be colored with the highest end of the palette
    'palette': ndbsi_palette
}

# These visualization settings define how the NDBSI change values are visualized
ndbsi_change_vis = {
    'min': -0.3,
    'max': 0.3,
    'palette': ndbsi_change_palette
}

# Create an interactive Map
map = geemap.Map(center=[25.07, 55.18], zoom=10)

# Add the Study Area as a Maplayer
map.addLayer(dubai.style(**{'color': 'black', 'width': 2}), {}, 'Study Area')

# Add the NDBSI Layer to the Map
map.addLayer(ndbsi_1995.clip(dubai), ndbsi_vis, 'NDBSI 1995')
map.addLayer(ndbsi_2005.clip(dubai), ndbsi_vis, 'NDBSI 2005')
map.addLayer(ndbsi_2015.clip(dubai), ndbsi_vis, 'NDBSI 2015')
map.addLayer(ndbsi_2024.clip(dubai), ndbsi_vis, 'NDBSI 2024')

# Add the Change Layer
map.addLayer(ndbsi_change.clip(dubai), ndbsi_change_vis, 'NDBSI Change')

# Display the Map
map


In [ ]:
years = [1995, 2005, 2015, 2024]
# It takes a list of 'ee.Image' objects as input and combines them into a single 'ee.ImageCollection' object.
ndbsi_collection = ee.ImageCollection([ndbsi_1995, ndbsi_2005, ndbsi_2015, ndbsi_2024])

In [ ]:
ndbsi_change_ = ee.ImageCollection([ndbsi_change])

In [ ]:
# A color palette for visualizing Bare soil area values.
ndbsi_palette = [
    '#2900fa',  
    '#1a9850',  
    '#ffffbf',  
    '#fb9c14',  
    '#d73027'   
]

# These visualization settings define how the BS values are mapped to the color palette
ndbsi_vis = {
    'min': -0.3,
    'max': 0.3,
    'palette': ndbsi_palette
}

In [ ]:
ndbsi_change_palette = [
    '#fb0114',  
    '#fbfdff',  
    '#039d02'   
]

ndbsi_change_vis = {
    'min': -0.3,
    'max': 0.3,
    'palette': ndbsi_change_palette
}

In [ ]:
# This dictionary 'video_args_ndsbi' contains the necessary parameters to generate a time-lapse video of the BA data for Dubai.
video_args_ndbsi = {
    'dimensions': 768,
    'region': dubai.geometry().bounds(),
    'framesPerSecond': 2,
    'min': ndbsi_vis['min'],
    'max': ndbsi_vis['max'],
    'palette': ndbsi_vis['palette'],
    'format': 'gif'
}

In [ ]:
video_args_ndbsi_change = {
    'dimensions': 768,
    'region': dubai.geometry().bounds(),
    'framesPerSecond': 2,
    'min': ndbsi_change_vis['min'],
    'max': ndbsi_change_vis['max'],
    'palette': ndbsi_change_vis['palette'],
    'format': 'gif'
}

In [ ]:
# specifies the output file path and name for the generated GIF video
out_gif_ndbsi = './ndbsi_selected_years.gif'
geemap.download_ee_video(ndbsi_collection, video_args_ndbsi, out_gif_ndbsi)

# This function takes an existing GIF video and adds text overlays to each frame.
geemap.add_text_to_gif(out_gif_ndbsi, './ndbsi_dubai_text.gif', text_sequence=years,
                       xy=('3%', '5%'), font_size=30, font_color='white', add_progress_bar=False, duration=500)

In [ ]:
out_gif_ndbsi_change = './ndbsi_change.gif'
geemap.download_ee_video(ndbsi_change_, video_args_ndbsi_change, out_gif_ndbsi_change)

geemap.add_text_to_gif(out_gif_ndbsi_change, './ndbsi_dubai_change.gif', text_sequence='Change Detection NDBSI',
                       xy=('45%', '5%'), font_size=30, font_color='white', add_progress_bar=False, duration=500)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.colors import LinearSegmentedColormap

ndbsi_palette = [
    '#2900fa',  
    '#1a9850',  
    '#ffffbf',  
    '#fb9c14',  
    '#d73027'   
]

# Creating a custom colormap from the list of colors
cmap_ndbsi = LinearSegmentedColormap.from_list("my_ndbsi_cmap", ndbsi_palette)

fig, ax = plt.subplots(figsize=(6, 1), facecolor='black') # This line creates a Matplotlib figure and an axes object.
gradient = np.linspace(-1, 1, 256).reshape(1, -1) # This line creates a 1D NumPy array named 'gradient' containing 256 evenly spaced values between -1 and 1 
ax.imshow(gradient, aspect='auto', cmap=cmap_ndbsi) 

# Re-enable axes
ax.set_axis_on()

# Set ticks and labels
ax.set_xticks([0, 255])  # Positions for the Ticks 
ax.set_xticklabels(['-1', '1'], color='white') # Labels for the Ticks


ax.set_yticks([])

# Remove top, right, and left axis spines
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.spines['bottom'].set_color('white') # This line sets the color of the bottom axis spine to white

# This line sets the color of the x-axis tick marks to white
ax.tick_params(axis='x', colors='white')

plt.savefig("legend_NDBSI.png", bbox_inches='tight', pad_inches=0)
plt.close()

In [ ]:
# Farbpalette für BSI-Änderung
ndbsi_change_palette = [
    '#fb0114',  # Rückgang (mehr Vegetation)
    '#fbfdff',  # Keine Veränderung
    '#039d02'   # Zunahme Bare Soil
]

# Creating a custom colormap from the list of colors
cmap_ndbsi_change = LinearSegmentedColormap.from_list("my_ndbsi_change_cmap", ndbsi_change_palette)

fig, ax = plt.subplots(figsize=(6, 1), facecolor='black') # This line creates a Matplotlib figure and an axes object.
gradient = np.linspace(-1, 1, 256).reshape(1, -1) # This line creates a 1D NumPy array named 'gradient' containing 256 evenly spaced values between -1 and 1 
ax.imshow(gradient, aspect='auto', cmap=cmap_ndbsi_change) 

# Re-enable axes
ax.set_axis_on()

# Set ticks and labels
ax.set_xticks([0, 255])  # Positions for the Ticks 
ax.set_xticklabels(['decrease', 'increase'], color='white') # Labels for the Ticks


ax.set_yticks([])

# Remove top, right, and left axis spines
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.spines['bottom'].set_color('white') # This line sets the color of the bottom axis spine to white

# This line sets the color of the x-axis tick marks to white
ax.tick_params(axis='x', colors='white')

plt.savefig("legend_NDBSI_Change.png", bbox_inches='tight', pad_inches=0)
plt.close()